In [1]:
import code_library as cl
from dotenv import dotenv_values

In [2]:
! pip install pandas 

In [3]:
secrets = dotenv_values(".env")

In [4]:
connection_parameters = {
"account":secrets['account'], 
"user":secrets['user'], 
"password": secrets['password'], 
"role":secrets['role'], 
"warehouse":"COMPUTE_WH", 
"database":"DEMO_DB", 
"schema":"PUBLIC" 
}

In [5]:
session = cl.snowconnection(connection_parameters)

## Testing the second function, copy_into_table

In [ ]:
config_file = {"Database_name":"DEMO_DB",\
"Schema_name":"PUBLIC",
"Target_table":"EMPLOYEE",
"Reject_table":"EMPLOYEE_REJECTS",
"target_columns":["FIRST_NAME","LAST_NAME","EMAIL","ADDRESS","CITY","DOJ"],
"on_error":"CONTINUE",
"Source_location":"@s3_stg/employee/",
"Source_file_type":"csv"
}

In [ ]:
def copy_to_table(session,config_file,schema='NA'):
    database_name = config_file.get("Database_name")
    Schema_name = config_file.get("Schema_name")
    Target_table = config_file.get("Target_table")
    target_columns = config_file.get("target_columns")
    on_error = config_file.get("on_error")
    Source_location = config_file.get("Source_location")

    if config_file.get("Source_file_type") == 'csv':
            schema = schema
            df = session.read.schema(schema).csv("'"+Source_location+"'")
        
    with session.query_history() as query_history:
        copied_into_result = df.copy_into_table(database_name+"."+Schema_name+"."+Target_table, target_columns=target_columns,force=True,on_error=on_error)
    query = query_history.queries
    # Mention command to collect query id of copy command executed.
    for id in query:
        if "COPY" in id.sql_text:
            qid = id.query_id
    return copied_into_result, qid